In [34]:
#Caricare le librerie
import tweepy
import pandas as pd
import numpy as np
import re
import json

In [35]:
consumer_key= "rrR0Znz98oKkjfB8b72oLH9Qc"
consumer_secret= "m7iOPX6iW8R3IalRf7x56YVeIb18o3I1bhUftg8JzlQ6fAjLgW"
access_token= "1442250607-3cJYKYyh7Ae19nMcv5YoxOIB2XlvgVX0ROaDSJx"
access_token_secret= "ZOnaaCTgO7DsKtmPjVruOlkvzBY8p6MbkKLjiYLdamBYE"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [36]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")
#NB: hasattr(obj, name, /) Return whether the object has an attribute with the given name.
hashtag="#lockdown"
list_tweet=[]
list_originals=[]
list_retweetted=[]
n=4500
for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,count=n,lang="it").items(n):

  #gestione dei retweet con try-except
  try:
    list_originals.append(tweet.retweeted_status.id) #creo lista dei retweets id 
    #lista tweets che sono stati retweettati
    list_retweetted.append([tweet.retweeted_status.created_at, tweet.retweeted_status.id,tweet.retweeted_status.user.id, tweet.retweeted_status.text, 
                     tweet.retweeted_status.favorite_count, tweet.retweeted_status.retweet_count, 
                     tweet.retweeted_status.user.screen_name]) 
  except:
    list_originals.append(0)
  #creo lista e df dei tweet  
  list_tweet.append([tweet.created_at, tweet.id,tweet.user.id, tweet.text, 
                     tweet.favorite_count, tweet.retweet_count, 
                     tweet.user.location,tweet.user.screen_name,
                     hasattr(tweet, "retweeted_status"),
                     tweet.in_reply_to_screen_name])

                     
tweets = pd.DataFrame(list_tweet[1:], 
                      columns=['date','id','user_id','text','like','n_rt','location','author',
                               "retweet","reply_name"])
tweets["retwitted_id"]=list_originals[1:]
#creo df dei retweet
retweetted = pd.DataFrame(list_retweetted[1:], 
                      columns=['date','id','author_id','text','like','n_rt','author'])                                                    
tweets.head()

,date,id,user_id,text,like,n_rt,location,author,retweet,reply_name,retwitted_id
0,2021-12-24 15:50:27+00:00,1474407128492347398,1063284314,RT @EsercitoCrucian: Noi invece Proponiamo il ...,0,676,Roma,romanino48,True,None,1472956905664520192
1,2021-12-24 15:50:25+00:00,1474407121596948497,1257213168541147138,Il #lockdown unica soluzione. Il Lockdown non ...,2,0,,Limbolimbo18,False,None,0
2,2021-12-24 15:50:21+00:00,1474407103272034305,2637882316,@cicciogia Perché fra poco iniziamo il secondo...,1,0,,CiccioniSe,False,cicciogia,0
3,2021-12-24 15:43:01+00:00,1474405256855531520,2483513455,RT @EsercitoCrucian: Noi invece Proponiamo il ...,0,676,,piccologori,True,None,1472956905664520192
4,2021-12-24 15:37:58+00:00,1474403985972940809,1217767250695946241,RT @EsercitoCrucian: Noi invece Proponiamo il ...,0,676,,clamar65,True,None,1472956905664520192


In [37]:
#selezione dei tweet più retwettati 
retweetted.sort_values("n_rt",ascending=False).drop_duplicates()

,date,id,author_id,text,like,n_rt,author
1024,2021-11-19 13:28:48+00:00,1461687908797227008,50989769,Mi sembra il giorno giusto per ripostare il fa...,2479,1331,darkskywriter
0,2021-12-20 15:47:47+00:00,1472956905664520192,2222183850,Noi invece Proponiamo il #Lockdown TOTALE e de...,899,676,EsercitoCrucian
1096,2021-12-18 19:09:39+00:00,1472282934073872397,3096871079,Che fate a Capodown?\n#lockdown,4772,671,lefrasidiosho
666,2021-12-18 22:56:11+00:00,1472339940327444482,2377487094,Con il 90% di #vaccinati si moltiplicano i pos...,2241,594,intuslegens
2560,2021-12-18 20:23:00+00:00,1472301392467476489,1323040160,Saranno intitolate strade e piazze in suo onor...,1413,484,strange_days_82
...,...,...,...,...,...,...,...
2552,2021-12-20 17:31:01+00:00,1472982887876210689,1466403465786277895,E noi rivendichiamo il diritto di dirti che se...,2,1,ROBERTOViVi1
2553,2021-12-20 17:33:45+00:00,1472983575339450381,1467094064449441794,La mia libertà non finisce dove cominciano le ...,0,1,Nobu_ViVi
1188,2021-12-20 17:28:46+00:00,1472982318868582405,1466753520598720512,Potete avere e conoscere tutte le ragioni per ...,2,1,Vsuperstar91
2555,2021-12-20 17:34:14+00:00,1472983695128674308,1467094064449441794,Se dici una bugia abbastanza grande e continui...,3,1,Nobu_ViVi


##### Vado a vedere quali sono i tweet più retwittati e quante volte sono stati ritwittati dalla nostra rete (usando SQL)

In [38]:
#Open the terminal and run-   pip install U pandasql
#Open your Jupyter Notebook and in any cell run-    !pip install -U pandasql
import pandasql as ps
q1 = """SELECT id,like,n_rt, author,count() FROM retweetted group by id order by n_rt desc"""
print(ps.sqldf(q1, locals()))

                      id  like  n_rt           author  count()
0    1461687908797227008  2479  1331    darkskywriter        1
1    1472956905664520192   899   676  EsercitoCrucian      521
2    1472282934073872397  4772   671    lefrasidiosho        8
3    1472339940327444482  2241   594      intuslegens       10
4    1472301392467476489  1413   484  strange_days_82       12
..                   ...   ...   ...              ...      ...
341  1472854224719552514     0     1   InformazioneOg        1
342  1472638825369382913     2     1    ContiMariapia        1
343  1472587980284043276    10     1  GiorgioCameran1        1
344  1472458878268985346     1     1        Radio1Rai        1
345  1470810332654362634     2     1   BorasoGianluca        1

[346 rows x 5 columns]


#### Creazione input Grephi tramite scrittura file .csv
Da rivedere:
- Abbiamo creato nodi con l'id del tweet, non per utente
- Si possono pesare gli archi tra utenti per il numero di retweet tra essi? Ha senso?

In [39]:
rt_tweets=tweets[tweets.retweet==True]

In [40]:
tweets_ids=rt_tweets[['user_id','author']]
retwitted_ids=retweetted[['author_id','author']].rename(columns={'author_id':'user_id'})
rt_tweets_ids=pd.concat([tweets_ids,retwitted_ids]).drop_duplicates().reset_index(drop=True)

In [44]:
Nodes_df=pd.DataFrame({'Id':rt_tweets_ids.user_id,'Label':rt_tweets_ids.author,'Interval':None})

In [45]:
Edges_df=pd.DataFrame({'Source':rt_tweets.user_id,'Target':rt_tweets.retwitted_id,'Type':'Directed','Kind':None,'Id':range(1, len(rt_tweets)+1),'Label':None,'Interval':None,'Weight':1.0})

In [46]:
Nodes_df.to_csv('Nodes_df.csv', index=False)
Edges_df.to_csv('Edges_df.csv', index=False)